<a href="https://colab.research.google.com/github/jyotidabass/Tongue_coating_detection_GLCM_features/blob/main/Tongue_coating_detection_GLCM_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import cv2
import pandas as pd
import numpy as np
import os
from skimage.feature.texture import greycomatrix
from skimage.feature.texture import greycoprops
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from tqdm import tqdm

In [29]:
def get_feature(img):
    
    img_graymatrix = greycomatrix(img, [1], [0, np.pi/2])
#     print("img_graymatrix shape: ",img_graymatrix.shape)
    
    img_contrast = greycoprops(img_graymatrix, 'contrast')
#     print("img_contrast shape: ", img_contrast.shape)
    
    img_homogeneity = greycoprops(img_graymatrix, 'homogeneity')
#     print("img_homogeneity shape: ", img_homogeneity.shape)
    
    img_correlation = greycoprops(img_graymatrix, 'correlation')
#     print("img_correlation shape: ", img_correlation.shape)
    
    
    img_contrast_flattened = img_contrast.flatten()
    img_homogeneity_flattened = img_homogeneity.flatten()
    img_correlation_flattened = img_correlation.flatten()
    
    features = np.concatenate([img_contrast_flattened, img_homogeneity_flattened, 
                             img_correlation_flattened])
#     print("final_feature shape: ",features.shape)
    
    return(features)

In [30]:
def feature_extraction(path_to_folder, class_label):
    data_list=[]
#     count=1
    for file_name in tqdm(os.listdir(path_to_folder)):
#         if(count>1):
#             break
        path_to_img = os.path.join(path_to_folder,file_name)
        img = cv2.imread(path_to_img, 0) # grayscale image
         
        if np.shape(img) == ():
            continue
        
        final_feature = get_feature(img)
#         print("final_feature shape: ",final_feature.shape)
#         print("final_feature is: ",final_feature)
        
        final_feature=list(final_feature)
        final_feature.insert(0,file_name)
        final_feature.insert(1,class_label)
        
        data_list.append(final_feature)
        
#         count+=1
       
    return(data_list)

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
Coated_tongue = '/content/drive/MyDrive/Two-class-coated-uncoated/Train/class0'
Non_coated_tongue = '/content/drive/MyDrive/Two-class-coated-uncoated/Train/class1'

data_list1 = feature_extraction(Coated_tongue, 0)
data_list2 = feature_extraction(Non_coated_tongue, 1)

df = pd.DataFrame(data_list1)
df = df.append(pd.DataFrame(data_list2), ignore_index=True)
# #  --------------------------------------------------------------------------------------


df.rename(columns = {0: "image_names", 1: "label"}, inplace = True)

100%|██████████| 234/234 [00:07<00:00, 32.56it/s]


In [35]:
df.head()

,image_names,label,2,3,4,5,6,7
0,0CA41A93-47F3-40CD-9479-8FB4CCFED190 - vinay k...,0,14.848787,13.461818,0.320131,0.319495,0.996106,0.996470
1,5E2724F4-504A-4820-8328-EBADDD42AE00 - G D siv...,0,5.107032,5.092409,0.489468,0.488068,0.997849,0.997859
2,8B58C21B-2441-480A-A1F1-2043488FF2AA - Sharu M...,0,10.707498,10.583648,0.348152,0.345095,0.997783,0.997807
3,8F28F524-4ADC-4D6A-B212-CD01C180EC0F - Anil Ch...,0,13.754650,12.786963,0.326545,0.329048,0.994658,0.995039
4,3610B754-C9DD-47BD-BCEF-5757A1EBF5FA - A_JAY D...,0,13.348309,12.031077,0.388370,0.395029,0.996723,0.997045


In [36]:
df.shape

(465, 8)

In [37]:
df.to_csv('features.csv', index=False)

SVM CLASSIFICATION

In [38]:
#df=pd.read_csv('features.csv')

In [39]:
array=df.values
x_feature=array[:,2:]
y_label=array[:,1].astype('int')
print(x_feature.shape)
print(y_label.shape)

(465, 6)
(465,)


In [40]:
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.10,random_state=7)

In [41]:
# Normalise the data after splitting to avoid information leak between train and test set.

scaler_norm = MinMaxScaler()

X_train = scaler_norm.fit_transform(X_train)
X_test = scaler_norm.fit_transform(X_test)

In [42]:
# Random check
model_SVC=SVC(kernel='rbf',C=100,gamma=0.001)

kfold=KFold(n_splits=10, shuffle=True)
cv_results=cross_val_score(model_SVC,X_train,Y_train,cv=kfold,scoring='accuracy')
msg="%s %f (%f)" % ('Training Accuracy: ',cv_results.mean(),cv_results.std())
print(msg)

Training Accuracy:  0.471370 (0.050600)


In [43]:
# SVM hypertuning using GridSeachCV
model_SVC=SVC()

kfold=KFold(n_splits=10)
param_grid = {'C': [1, 10, 100, 500, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

grid=GridSearchCV(estimator=model_SVC,param_grid=param_grid,scoring='accuracy',cv=kfold,verbose=3)
grid_result=grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV 1/10] END .........C=1, gamma=1, kernel=rbf;, score=0.476 total time=   0.0s
[CV 2/10] END .........C=1, gamma=1, kernel=rbf;, score=0.381 total time=   0.0s
[CV 3/10] END .........C=1, gamma=1, kernel=rbf;, score=0.333 total time=   0.0s
[CV 4/10] END .........C=1, gamma=1, kernel=rbf;, score=0.571 total time=   0.0s
[CV 5/10] END .........C=1, gamma=1, kernel=rbf;, score=0.381 total time=   0.0s
[CV 6/10] END .........C=1, gamma=1, kernel=rbf;, score=0.500 total time=   0.0s
[CV 7/10] END .........C=1, gamma=1, kernel=rbf;, score=0.524 total time=   0.0s
[CV 8/10] END .........C=1, gamma=1, kernel=rbf;, score=0.524 total time=   0.0s
[CV 9/10] END .........C=1, gamma=1, kernel=rbf;, score=0.439 total time=   0.0s
[CV 10/10] END ........C=1, gamma=1, kernel=rbf;, score=0.585 total time=   0.0s
[CV 1/10] END .......C=1, gamma=0.1, kernel=rbf;, score=0.452 total time=   0.0s
[CV 2/10] END .......C=1, gamma=0.1, kernel=rb

In [44]:
model_SVC = SVC(C=500,gamma=0.1, kernel='rbf')
model_SVC.fit(X_train,Y_train) 

predictions=model_SVC.predict(X_test)

print(accuracy_score(Y_test,predictions))
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

0.5531914893617021
[[16  3]
 [18 10]]
              precision    recall  f1-score   support

           0       0.47      0.84      0.60        19
           1       0.77      0.36      0.49        28

    accuracy                           0.55        47
   macro avg       0.62      0.60      0.55        47
weighted avg       0.65      0.55      0.53        47

